In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp
import cvxpy as cp
import json
import hashlib
import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky
import xxhash
from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem, DoubleTraceOperator
from bmn.linear_algebra import get_null_space_dense, create_sparse_matrix_from_dict, get_row_space_dense, get_null_space_sparse
from bmn.bootstrap import BootstrapSystem
from bmn.solver_trustregion import (
    minimal_eigval,
    sdp_init, sdp_relax,
    sdp_minimize,
    #minimize,
    get_quadratic_constraint_vector_sparse,
    get_quadratic_constraint_vector_dense,
    compute_L2_norm_of_quadratic_constraints,
)
from bmn.solver_pytorch import solve_bootstrap as solve_bootstrap_pytorch
from bmn.models import OneMatrix, TwoMatrix, MiniBFSS
from bmn.brezin import compute_Brezin_energy, compute_Brezin_energy_Han_conventions
import os
from bmn.debug_utils import disable_debug

import yaml
from bmn.config_utils import (
    run_bootstrap_from_config,
    generate_configs_one_matrix,
    generate_configs_two_matrix,
    generate_configs_three_matrix,
    generate_configs_bfss,
    )

from bmn.models import OneMatrix, TwoMatrix, MiniBFSS
import pandas as pd


# plot settings
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler
import torch
import torch.optim as optim
from torch.nn import ReLU

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['lines.linewidth'] = 2
plt.rc('font', family='serif',size=16)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=16)
matplotlib.rcParams['axes.prop_cycle'] = cycler(
    color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
    )
matplotlib.rcParams.update(
    {"axes.grid":True,
    "grid.alpha":0.75,
    "grid.linewidth":0.5}
    )
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

np.set_printoptions(linewidth=120)  # Adjust the number to the desired width
torch.set_printoptions(linewidth=120)  # Adjust the number to the desired width

In [ ]:
L = 4
model = MiniBFSS(couplings={"lambda": 1})
checkpoint_path = f"checkpoints/MiniBFSS_L_{L}_lazy_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=L,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path,
    odd_degree_vanish=True,
    #load_from_previously_computed=True
    )
bootstrap.load_constraints(checkpoint_path)

In [ ]:
x3 = 0.5664224028587341
x4 = 0.5642986304623219
x3 - x4

In [ ]:
len(bootstrap.linear_constraints)

In [ ]:
new_constraints = []

set_of_hashes = set()
for st_operator in bootstrap.linear_constraints:
    new_dict = {}
    norm = 1
    for idx, (k, v) in enumerate(st_operator):
        if idx == 0:
            norm = float(1/v)
        new_dict[''.join(k)] = float(v) * norm
    #new_dict = {''.join(k): float(v) for k, v in st_operator}
    s = json.dumps(new_dict).encode('utf-8')
    hash_digest = hashlib.md5(s).digest()
    if hash_digest not in set_of_hashes:
        new_constraints.append(st_operator)
        set_of_hashes.add(hash_digest)

In [ ]:
print(f"number of unique hashes: {len(set_of_hashes)}, ratio of hashes to constraints: {len(set_of_hashes) / len(bootstrap.linear_constraints)}")

In [ ]:
bootstrap.linear_constraints = new_constraints

In [ ]:
linear_constraint_matrix = bootstrap.build_linear_constraints()

In [ ]:
linear_constraint_matrix.shape

In [ ]:
bootstrap.build_null_space_matrix()